## X,Y 좌표로 자치구/행정동 추출하기

In [13]:
import geopandas as gpd
import pandas as pd

In [18]:
# 공간 데이터 csv로 변환하여 저장
gdf = gpd.read_file("/Users/kimjisu/Desktop/seoulbigcam/Raw Data/2020/TL_TBVIATR_FCLTY_INFO_2020.shp")
csv_file = "/Users/kimjisu/Desktop/seoulbigcam/Preprocessed Data/서울시 주요시설과 집객시설 공간데이터(2020).csv"
gdf.to_csv(csv_file, index=False)


## 생략가능 아래 코드만 실행하면 됩니다

In [28]:
# 엑셀을 이용해 가볍게 1차 전처리 한 파일을 불러오기

seoul = pd.read_csv('Preprocessed Data/서울시 주요시설과 집객시설 공간데이터(2020).csv')
seoul.head()

,STDR_YM,VIAT_SE_CD,VIAT_CD,VIAT_ID,LCLASDC,MLSFCDC,SCLASDC,DCLASDC,BCLASDC,VIAT_NM,...,ADRES_NM,TELNO,X_VALUE,Y_VALUE,ADRES_SP,BLCK_CD,REGIST_DE,ERSR_DE,REGIST_DT,geometry
0,NaN,30102,200,AA0000000001,교육/보건,교육시설,초등교육기관,공립초등학교,NaN,"강덕초교,고덕그라시움버스정류장쉘터",...,NaN,NaN,214388.672913,551205.841919,NaN,156656.0,NaN,NaN,NaN,POINT (214388.67291259766 551205.8419189453)
1,NaN,70301,302,AA0000000002,산업,서비스산업,종합상품판매업,대형상가,일반종합상가,고덕그라시움상가,...,NaN,NaN,214555.910095,550962.356812,NaN,156617.0,NaN,NaN,NaN,POINT (214555.91009521484 550962.3568115234)
2,NaN,30102,200,AA0000000003,교육/보건,교육시설,초등교육기관,공립초등학교,NaN,서울강덕초등학교,...,NaN,NaN,214224.118713,550979.324829,NaN,156670.0,NaN,NaN,NaN,POINT (214224.1187133789 550979.3248291016)
3,NaN,30101,100,AA0000000004,교육/보건,교육시설,유아교육기관,어린이집,NaN,은빛어린이집,...,NaN,NaN,214171.512024,550841.469116,NaN,155226.0,NaN,NaN,NaN,POINT (214171.51202392578 550841.4691162109)
4,NaN,70301,302,AA0000000005,산업,서비스산업,종합상품판매업,대형상가,일반종합상가,고덕그라시움상가,...,NaN,NaN,214481.776611,550819.438110,NaN,155064.0,NaN,NaN,NaN,POINT (214481.77661132812 550819.4381103516)


In [29]:
# 교육시설 제외하기

seoul = seoul[seoul['MLSFCDC'] != '교육시설']
seoul = seoul.reset_index(drop=True)
seoul.head()

,STDR_YM,VIAT_SE_CD,VIAT_CD,VIAT_ID,LCLASDC,MLSFCDC,SCLASDC,DCLASDC,BCLASDC,VIAT_NM,...,ADRES_NM,TELNO,X_VALUE,Y_VALUE,ADRES_SP,BLCK_CD,REGIST_DE,ERSR_DE,REGIST_DT,geometry
0,NaN,70301,302,AA0000000002,산업,서비스산업,종합상품판매업,대형상가,일반종합상가,고덕그라시움상가,...,NaN,NaN,214555.910095,550962.356812,NaN,156617.0,NaN,NaN,NaN,POINT (214555.91009521484 550962.3568115234)
1,NaN,70301,302,AA0000000005,산업,서비스산업,종합상품판매업,대형상가,일반종합상가,고덕그라시움상가,...,NaN,NaN,214481.776611,550819.438110,NaN,155064.0,NaN,NaN,NaN,POINT (214481.77661132812 550819.4381103516)
2,NaN,70301,302,AA0000000006,산업,서비스산업,종합상품판매업,대형상가,일반종합상가,그라시움상가,...,NaN,NaN,214465.779480,551177.935486,NaN,156674.0,NaN,NaN,NaN,POINT (214465.77947998047 551177.9354858398)
3,NaN,60203,200,AA0000000007,레저/관광/예술,문화/예술시설,박물관,공립박물관,NaN,송파BOOK책박물관,...,NaN,NaN,209236.807129,544386.432007,NaN,154275.0,NaN,NaN,NaN,POINT (209236.80712890625 544386.4320068359)
4,NaN,70301,400,AA0000000011,산업,서비스산업,종합상품판매업,시장,미분류,"가락시장,헬리오시티버스정류장쉘터",...,NaN,NaN,209386.103088,543972.747620,NaN,154048.0,NaN,NaN,NaN,POINT (209386.1030883789 543972.7476196289)


In [30]:
# X,Y좌표 경도 위도로 변환하기

gdf = gpd.GeoDataFrame(seoul, geometry=gpd.points_from_xy(seoul.X_VALUE, seoul.Y_VALUE))

gdf.crs = 'epsg:5186'
gdf = gdf.to_crs('epsg:4326')

# 변환 좌표 속성 생성 
gdf['x'] = gdf['geometry'].x
gdf['y'] = gdf['geometry'].y

# 필요한 컬럼만 추출해서 저장하기
gdf = gdf[['MLSFCDC', 'VIAT_NM', 'x', 'y']]

print(gdf)

       MLSFCDC            VIAT_NM           x          y
0        서비스산업           고덕그라시움상가  127.164740  37.558074
1        서비스산업           고덕그라시움상가  127.163898  37.556787
2        서비스산업             그라시움상가  127.163724  37.560018
3      문화/예술시설         송파BOOK책박물관  127.104457  37.498893
4        서비스산업  가락시장,헬리오시티버스정류장쉘터  127.106140  37.495165
...        ...                ...         ...        ...
12985    서비스산업                NaN  126.894531  37.472180
12986     보건시설                NaN  126.909474  37.450467
12987    서비스산업                NaN  126.902265  37.471291
12988     보건시설                NaN  126.872023  37.524689
12989     보건시설                NaN  127.033093  37.527229

[12990 rows x 4 columns]


### 경도, 위도를 이용해 REST API로 행정동/자치구 추출하기

In [22]:
import pandas as pd
import requests 
import sys
import json
import datetime 

In [31]:
import pandas as pd
import requests

locations = []

def get_location(row):
    url = 'https://dapi.kakao.com/v2/local/geo/coord2regioncode.json?x={}&y={}'.format(row['x'], row['y'])
    headers = {
        "Authorization": "KakaoAK 8cd64c4299a983cac6fd3edef103f134"
    }
    response = requests.get(url, headers=headers)
    
    try:
        response_json = response.json()
        place = response_json['documents'][0]
        city = place.get('region_1depth_name')
        gu = place.get('region_2depth_name')
        dong = place.get('region_3depth_name')
        locations.append({'city': city, 'gu': gu, 'dong': dong})
    except (IndexError, AttributeError, KeyError):
        print('API 호출 오류:', response.text)


# 데이터프레임의 각 행에 대해 get_location 함수 호출
for i, row in gdf.iterrows():
    get_location(row)
    
    # 200개씩 처리될 때 메시지 출력
    if i % 100 == 0:
        print('{} 번째 경위도 좌표계에 대한 정보 불러오는 중...'.format(i))

# 결과를 데이터프레임에 추가
gdf['city'] = [location.get('city') for location in locations]
gdf['gu'] = [location.get('gu') for location in locations]
gdf['dong'] = [location.get('dong') for location in locations]


0 번째 경위도 좌표계에 대한 정보 불러오는 중...
100 번째 경위도 좌표계에 대한 정보 불러오는 중...
200 번째 경위도 좌표계에 대한 정보 불러오는 중...
300 번째 경위도 좌표계에 대한 정보 불러오는 중...
400 번째 경위도 좌표계에 대한 정보 불러오는 중...
500 번째 경위도 좌표계에 대한 정보 불러오는 중...
600 번째 경위도 좌표계에 대한 정보 불러오는 중...
700 번째 경위도 좌표계에 대한 정보 불러오는 중...
800 번째 경위도 좌표계에 대한 정보 불러오는 중...
900 번째 경위도 좌표계에 대한 정보 불러오는 중...
1000 번째 경위도 좌표계에 대한 정보 불러오는 중...
1100 번째 경위도 좌표계에 대한 정보 불러오는 중...
1200 번째 경위도 좌표계에 대한 정보 불러오는 중...
1300 번째 경위도 좌표계에 대한 정보 불러오는 중...
1400 번째 경위도 좌표계에 대한 정보 불러오는 중...
1500 번째 경위도 좌표계에 대한 정보 불러오는 중...
1600 번째 경위도 좌표계에 대한 정보 불러오는 중...
1700 번째 경위도 좌표계에 대한 정보 불러오는 중...
1800 번째 경위도 좌표계에 대한 정보 불러오는 중...
1900 번째 경위도 좌표계에 대한 정보 불러오는 중...
2000 번째 경위도 좌표계에 대한 정보 불러오는 중...
2100 번째 경위도 좌표계에 대한 정보 불러오는 중...
2200 번째 경위도 좌표계에 대한 정보 불러오는 중...
2300 번째 경위도 좌표계에 대한 정보 불러오는 중...
2400 번째 경위도 좌표계에 대한 정보 불러오는 중...
2500 번째 경위도 좌표계에 대한 정보 불러오는 중...
2600 번째 경위도 좌표계에 대한 정보 불러오는 중...
2700 번째 경위도 좌표계에 대한 정보 불러오는 중...
2800 번째 경위도 좌표계에 대한 정보 불러오는 중...
2900 번째 경위도 좌표계에 대한 정보 불러오는 중...
3000 번째 경위도 좌표계에 대한 정보

ConnectionError: ('Connection aborted.', OSError(65, 'No route to host'))

In [ ]:
# csv로 저장하기
gdf.to_csv("/Users/kimjisu/Desktop/seoulbigcam/Preprocessed Data/서울시 주요시설과 집객시설 공간데이터(2020)_1차.csv")

In [30]:
gdf_new = gdf[['MLSFCDC', 'gu', 'dong']]
gdf_dong = gdf.groupby(['gu', 'dong'], as_index=False)['MLSFCDC'].count()
gdf_dong

,gu,dong,MLSFCDC
0,강남구,개포1동,3
1,강남구,개포2동,9
2,강남구,개포3동,3
3,강남구,개포4동,20
4,강남구,논현1동,88
...,...,...,...
451,하남시,감일동,3
452,하남시,미사1동,1
453,하남시,미사2동,1
454,하남시,미사3동,2


In [32]:
# 행정동별 면적 데이터 로드

area = pd.read_csv("/Users/kimjisu/Desktop/seoulbigcam/Raw Data/행정동별 면적.csv")


area = area.drop(area.index[0:3])     # 필요없는 데이터 정리
area = area.reset_index(drop=True)    # 인덱스 리셋
area = area.drop('동별(1)', axis=1)    # 필요없는 컬럼 삭제
area = area.rename(columns={'동별(2)': 'gu',
                            '동별(3)':'dong',
                            '2022':'area_km2'})     # 컬럼명 변경

area.head()

,gu,dong,area_km2
0,종로구,소계,23.91
1,종로구,사직동,1.23
2,종로구,삼청동,1.49
3,종로구,부암동,2.27
4,종로구,평창동,8.87


In [22]:
# 데이터 병합

merge_gdf = gdf_dong.merge(area[['dong', 'area_km2']], on='dong', how = 'left')
merge_gdf

,gu,dong,MLSFCDC,area_km2
0,강남구,개포1동,3,1.27
1,강남구,개포2동,9,2.51
2,강남구,개포3동,3,1.24
3,강남구,개포4동,20,1.49
4,강남구,논현1동,88,1.25
...,...,...,...,...
453,하남시,감일동,3,NaN
454,하남시,미사1동,1,NaN
455,하남시,미사2동,1,NaN
456,하남시,미사3동,2,NaN


In [28]:
merge_gdf[merge_gdf['area_km2'].isna()]

,gu,dong,MLSFCDC,area_km2
75,고양시 덕양구,화전동,1,NaN
76,고양시 덕양구,효자동,1,NaN
77,과천시,과천동,2,NaN
100,광명시,광명1동,8,NaN
101,광명시,광명2동,31,NaN
102,광명시,광명3동,16,NaN
103,광명시,광명5동,3,NaN
104,광명시,광명6동,5,NaN
105,광명시,소하1동,9,NaN
106,광명시,철산1동,7,NaN


In [29]:
area.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 451 entries, 0 to 450
Data columns (total 3 columns):
 #   Column    Non-Null Count  Dtype 
---  ------    --------------  ----- 
 0   gu        451 non-null    object
 1   dong      451 non-null    object
 2   area_km2  451 non-null    object
dtypes: object(3)
memory usage: 10.7+ KB
